# New York AirBnB

__Import statements__

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import (
    train_test_split,
    cross_validate
)

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    FunctionTransformer
)
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn import datasets, ensemble
from catboost import CatBoostRegressor
from lightgbm.sklearn import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn import set_config
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import VotingRegressor

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

import eli5

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import PolynomialFeatures

### Importing the dataset

In [88]:
data = pd.read_csv("../data/raw/airbnb.csv")

/Users/valliakella/opt/miniconda3/envs/abnb/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [89]:
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,30,48,2019-11-04,0.33,3,322,0,NaN
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",4869,LisaRoxanne,Brooklyn,Bedford-Stuyvesant,40.68494,-73.95765,Entire home/apt,73,1,408,2021-06-29,4.91,1,220,38,NaN
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2016-06-05,0.53,2,365,0,NaN
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,5,2,2021-08-08,0.02,1,91,1,NaN
4,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,505,2021-10-20,3.70,1,218,31,NaN


In [90]:
# Drop license, id, last_review
# data.drop(columns=["id", "license", "last_review"], inplace=True)

In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37713 entries, 0 to 37712
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              37713 non-null  int64  
 1   name                            37700 non-null  object 
 2   host_id                         37713 non-null  int64  
 3   host_name                       37682 non-null  object 
 4   neighbourhood_group             37713 non-null  object 
 5   neighbourhood                   37713 non-null  object 
 6   latitude                        37713 non-null  float64
 7   longitude                       37713 non-null  float64
 8   room_type                       37713 non-null  object 
 9   price                           37713 non-null  int64  
 10  minimum_nights                  37713 non-null  int64  
 11  number_of_reviews               37713 non-null  int64  
 12  last_review                     

In [92]:
data.dropna(subset=["reviews_per_month"], inplace=True)

In [93]:
data["last_review_year"] = pd.DatetimeIndex(data["last_review"]).year
data["last_review_month"] = pd.DatetimeIndex(data["last_review"]).month
data["last_review_date"] = pd.DatetimeIndex(data["last_review"]).day

In [94]:
# Splitting the data

In [95]:
train_val_data, test_data = train_test_split(data, test_size=0.2, random_state=123)
train_data, val_data = train_test_split(train_val_data, test_size=0.25, random_state=123)

In [97]:
## Some Stupidity

# def find_no_of_listings(data):
#     temp_df = pd.DataFrame(data["host_id"].value_counts()).reset_index()

#     temp_df.rename(columns={
#         "index": "host_id",
#         "host_id": "number_of_listings"
#     }, inplace=True)

#     data = pd.merge(data, temp_df, on="host_id")

#     data["number_of_listings"] = pd.cut(
#         data.number_of_listings,
#         bins=[0, 1, 10, np.inf],
#         labels=["1", "2-10", ">10"]
#     )

#     return data

In [98]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17100 entries, 19360 to 13967
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              17100 non-null  int64  
 1   name                            17098 non-null  object 
 2   host_id                         17100 non-null  int64  
 3   host_name                       17094 non-null  object 
 4   neighbourhood_group             17100 non-null  object 
 5   neighbourhood                   17100 non-null  object 
 6   latitude                        17100 non-null  float64
 7   longitude                       17100 non-null  float64
 8   room_type                       17100 non-null  object 
 9   price                           17100 non-null  int64  
 10  minimum_nights                  17100 non-null  int64  
 11  number_of_reviews               17100 non-null  int64  
 12  last_review                 

In [99]:
# X_train["neighbourhood"].value_counts()#.to_csv("apple.csv")

In [100]:
# # Converting the neighbourhoods occuring less than 86 times to "Others"
# less_frequent_list = X_train["neighbourhood"].value_counts()[86:].index.tolist()
# X_train.loc[X_train.query(
#     "neighbourhood in @less_frequent_list"
# ).index, "neighbourhood"] = "Others"

In [101]:
# X_train["neighbourhood"].value_counts()

In [102]:
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,last_review_year,last_review_month,last_review_date
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,...,48,2019-11-04,0.33,3,322,0,NaN,2019,11,4
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",4869,LisaRoxanne,Brooklyn,Bedford-Stuyvesant,40.68494,-73.95765,Entire home/apt,73,...,408,2021-06-29,4.91,1,220,38,NaN,2021,6,29
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,...,50,2016-06-05,0.53,2,365,0,NaN,2016,6,5
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,...,2,2021-08-08,0.02,1,91,1,NaN,2021,8,8
4,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Midtown,40.76457,-73.98317,Private room,68,...,505,2021-10-20,3.70,1,218,31,NaN,2021,10,20


In [103]:
numeric_features = [
    "latitude",
    "longitude",
    "price",
    "minimum_nights",
    "calculated_host_listings_count",
    "availability_365",
    "last_review_year",
    "last_review_date",
    "id",
    "host_id",
    "number_of_reviews",
    "number_of_reviews_ltm"
]

categorical_features = [
    "neighbourhood_group",
    "room_type",
    "last_review_month"
]

text_features = [
    "name"
]

drop_features = [
    "license",
    "host_name",
    "last_review",
    "neighbourhood"
]

passthrough_features = [

]

target_column = "reviews_per_month"

assert (
    len(numeric_features) +
    len(categorical_features) +
    len(text_features) +
    len(drop_features) +
    len(passthrough_features) +
    len([target_column])
) == len(train_data.columns)

In [123]:
X_train, y_train = train_data.drop(columns=[target_column]), train_data[target_column]
X_val, y_val = val_data.drop(columns=[target_column]), val_data[target_column]
X_train_big, y_train_big = train_val_data.drop(columns=[target_column]), train_val_data[target_column]

In [105]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17100 entries, 19360 to 13967
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              17100 non-null  int64  
 1   name                            17098 non-null  object 
 2   host_id                         17100 non-null  int64  
 3   host_name                       17094 non-null  object 
 4   neighbourhood_group             17100 non-null  object 
 5   neighbourhood                   17100 non-null  object 
 6   latitude                        17100 non-null  float64
 7   longitude                       17100 non-null  float64
 8   room_type                       17100 non-null  object 
 9   price                           17100 non-null  int64  
 10  minimum_nights                  17100 non-null  int64  
 11  number_of_reviews               17100 non-null  int64  
 12  last_review                 

In [106]:
function_transformer = FunctionTransformer(
    np.reshape, kw_args={"newshape": -1}
)

pipe_text_feats = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    function_transformer,
    CountVectorizer(stop_words="english", max_features=30)
)

column_transformer = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(handle_unknown="ignore", sparse=False), categorical_features),
    (pipe_text_feats, text_features),
    ("drop", drop_features),
    ("passthrough", passthrough_features)
)

In [107]:
column_transformer.fit(X_train)

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['latitude', 'longitude', 'price',
                                  'minimum_nights',
                                  'calculated_host_listings_count',
                                  'availability_365', 'last_review_year',
                                  'last_review_date', 'id', 'host_id',
                                  'number_of_reviews',
                                  'number_of_reviews_ltm']),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 ['neighbourhoo...
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('functiontransformer',
               

In [108]:
new_columns = (
    numeric_features +
    column_transformer.named_transformers_["onehotencoder"].get_feature_names_out().tolist() +
    column_transformer.named_transformers_["pipeline"].named_steps["countvectorizer"].get_feature_names_out().tolist() +
    passthrough_features
)

In [109]:
pd.DataFrame(column_transformer.transform(X_train), columns=new_columns)

,latitude,longitude,price,minimum_nights,calculated_host_listings_count,availability_365,last_review_year,last_review_date,id,host_id,...,new,nyc,park,private,room,spacious,studio,sunny,village,williamsburg
0,-0.406307,0.111532,-0.088612,0.399872,-0.259772,-0.237283,0.764204,-0.942082,0.240064,-0.705988,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.960255,-0.597747,-0.051700,-0.550828,-0.259772,-0.903438,0.764204,1.002941,1.336932,-0.740110,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.122256,-0.881149,-0.002484,-0.664912,5.811509,-0.266247,0.764204,0.183984,1.530824,0.467355,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.575909,-0.306592,-0.113220,0.399872,-0.155693,-0.135912,0.764204,0.286353,1.274427,2.118255,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.913452,1.029390,-0.338795,-0.626884,-0.190386,0.204406,0.764204,-1.453930,-0.186715,0.087669,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17095,1.229243,-0.304653,-0.379808,-0.702940,-0.259772,-0.903438,0.764204,-0.839712,1.434540,-0.406440,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
17096,0.790320,0.746570,-0.404416,0.323816,-0.225079,-0.686214,0.764204,0.695832,1.487830,2.372057,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
17097,-0.617706,0.056480,-0.474139,-0.702940,0.052466,1.167433,0.764204,0.491093,0.817329,-0.768836,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
17098,1.412290,-0.017181,-0.310085,-0.626884,-0.259772,-0.816548,-0.314539,0.183984,0.818855,-0.257824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
# Credits to Varada K.

# def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
#     """
#     Returns mean and std of cross validation

#     Parameters
#     ----------
#     model :
#         scikit-learn model
#     X_train : numpy array or pandas DataFrame
#         X in the training data
#     y_train :
#         y in the training data

#     Returns
#     ----------
#         pandas Series with mean scores from cross_validation
#     """

#     scores = cross_validate(model, X_train, y_train, **kwargs)

#     mean_scores = pd.DataFrame(scores).mean()
#     std_scores = pd.DataFrame(scores).std()
#     out_col = []

#     for i in range(len(mean_scores)):
#         out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

#     return pd.Series(data=out_col, index=mean_scores.index)

In [111]:
def cross_val_scores(model, X_train, y_train, X_val, y_val, return_train_score):

    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)

    score_dict = {
        "r2_train": model.score(X_train, y_train),
        "r2_test": model.score(X_val, y_val),
        "mse_train": mean_squared_error(y_train, y_train_pred),
        "mse_test": mean_squared_error(y_val, y_val_pred),
        "mape_train": mean_absolute_percentage_error(y_train, y_train_pred),
        "mape_test": mean_absolute_percentage_error(y_val, y_val_pred)
    }
    scores_result = pd.Series(score_dict)
    return scores_result

In [112]:
# Adapted from Lecture Notes

def mape(true, pred):
    return 100.0 * np.mean(np.abs((pred - true) / true))


# make a scorer function that we can pass into cross-validation
mape_scorer = make_scorer(mape, greater_is_better=False)

scoring_metrics = {
    "neg RMSE": "neg_root_mean_squared_error",
    "r2": "r2",
    "mape": mape_scorer,
}

### Baseline Regression

In [113]:
results = {}

In [114]:
models = {
    "Dummy": make_pipeline(
        column_transformer, DummyRegressor()
    ),
    "Ridge": make_pipeline(
        column_transformer, Ridge()
    ),
    "Lasso": make_pipeline(
        column_transformer, Lasso()
    ),
    "Random_Forest_reg": make_pipeline(
        column_transformer, RandomForestRegressor(random_state=123)
    ),
    "XGBoost_reg": make_pipeline(
        column_transformer, XGBRegressor(verbosity=0)
    ),
    "lgbm_reg": make_pipeline(
        column_transformer, LGBMRegressor()
    ),
    "catBoost_reg": make_pipeline(
        column_transformer, CatBoostRegressor(verbose=0)
    )
}

for model_name, model in models.items():
    results[model_name] = cross_val_scores(
        model,
        X_train,
        y_train,
        X_val,
        y_val,
        return_train_score=True
    )

In [115]:
pd.DataFrame(results)

,Dummy,Ridge,Lasso,Random_Forest_reg,XGBoost_reg,lgbm_reg,catBoost_reg
r2_train,0.000000,0.389500,0.307888,0.926435,0.958081,0.795359,0.875625
r2_test,-0.000010,0.484823,0.407891,0.500858,0.433224,0.504185,0.553416
mse_train,18.968049,11.579991,13.128019,1.395379,0.795113,3.881636,2.359146
mse_test,17.962028,9.253537,10.635373,8.965511,10.180341,8.905761,8.021479
mape_train,14.434554,4.038569,9.018054,0.144263,0.603201,0.669306,0.841592
mape_test,14.117369,3.872633,8.823251,0.413916,0.915507,0.704105,0.916343


## Feature Selection


In [116]:
from sklearn.feature_selection import SequentialFeatureSelector

results_cat_boost={}
n_features= [30, 35, 40, 45, 50, 55]
for i in n_features:
    pipe_forward = make_pipeline(
        column_transformer,
        SequentialFeatureSelector(Lasso(), direction="forward", n_features_to_select=i),
        CatBoostRegressor(verbose=0)
    )

    results_cat_boost[f"rf_forward_fs_{i}"] = cross_val_scores(pipe_forward, X_train, y_train, X_val, y_val, return_train_score=True)


In [117]:
pd.DataFrame(results_cat_boost)

,rf_forward_fs_30,rf_forward_fs_35,rf_forward_fs_40,rf_forward_fs_45,rf_forward_fs_50,rf_forward_fs_55
r2_train,0.868422,0.873455,0.878480,0.877222,0.871206,0.875903
r2_test,0.529463,0.537814,0.543390,0.522995,0.541635,0.536853
mse_train,2.495782,2.400321,2.304990,2.328854,2.442979,2.353875
mse_test,8.451725,8.301721,8.201566,8.567896,8.233088,8.318970
mape_train,0.806158,0.766644,0.798841,0.817904,0.889809,0.907281
mape_test,0.904520,0.869985,0.866573,0.880239,0.969967,1.012950


## Polynomial Features Regression

In [120]:
deg = 2
poly_feats = PolynomialFeatures(degree=deg)
pipe_poly = make_pipeline(column_transformer, poly_feats, Ridge())
results["poly_feat"]= cross_val_scores(
    pipe_poly,
    X_train,
    y_train,
    X_val,
    y_val,
    return_train_score=True 
)

In [121]:
pd.DataFrame(results)

,Dummy,Ridge,Lasso,Random_Forest_reg,XGBoost_reg,lgbm_reg,catBoost_reg,poly_feat
r2_train,0.000000,0.389500,0.307888,0.926435,0.958081,0.795359,0.875625,0.550253
r2_test,-0.000010,0.484823,0.407891,0.500858,0.433224,0.504185,0.553416,0.498116
mse_train,18.968049,11.579991,13.128019,1.395379,0.795113,3.881636,2.359146,8.530831
mse_test,17.962028,9.253537,10.635373,8.965511,10.180341,8.905761,8.021479,9.014759
mape_train,14.434554,4.038569,9.018054,0.144263,0.603201,0.669306,0.841592,4.635072
mape_test,14.117369,3.872633,8.823251,0.413916,0.915507,0.704105,0.916343,5.197266


### Hyperparameter Tuning

#### Hyperparam tune the Ridge

In [79]:
param_grid = {'ridge__alpha': 10.0 ** np.arange(-2, 6, 1)}

In [124]:
pipe_lr = make_pipeline(column_transformer, Ridge())

random_search = RandomizedSearchCV(
    pipe_lr,
    param_grid,
    n_jobs=-1,
    return_train_score=True,
    scoring=scoring_metrics,
    refit='r2'
)

In [125]:
random_search.fit(X_train_big, y_train_big)

/Users/valliakella/opt/miniconda3/envs/abnb/lib/python3.9/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('standardscaler',
                                                                               StandardScaler(),
                                                                               ['latitude',
                                                                                'longitude',
                                                                                'price',
                                                                                'minimum_nights',
                                                                                'calculated_host_listings_count',
                                                                                'availability_365',
                                                                                'last_review_year',
                                                     

In [126]:
pd.DataFrame(random_search.cv_results_)[[
        "mean_fit_time", 
        "mean_score_time", 
        "param_ridge__alpha", 
        "mean_train_neg RMSE", 
        "std_train_neg RMSE",
        "mean_test_mape",
        "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by='mean_test_r2', ascending=False)

,mean_fit_time,mean_score_time,param_ridge__alpha,mean_train_neg RMSE,std_train_neg RMSE,mean_test_mape,mean_train_mape,mean_test_r2,mean_train_r2
4,0.206660,0.038449,100.0,-3.312132,0.131516,-385.279139,-384.468327,0.409383,0.413215
3,0.205309,0.056504,10.0,-3.311648,0.131473,-393.094137,-391.997237,0.409119,0.413386
2,0.207704,0.048118,1.0,-3.311638,0.131472,-394.137128,-393.012350,0.409064,0.413390
1,0.235467,0.053459,0.1,-3.311638,0.131472,-394.244897,-393.118176,0.409058,0.413390
0,0.280439,0.056993,0.01,-3.311638,0.131472,-394.255704,-393.128797,0.409057,0.413390
5,0.191880,0.041282,1000.0,-3.319000,0.131689,-364.191336,-364.085037,0.408723,0.410781
6,0.193484,0.042094,10000.0,-3.435531,0.133703,-389.226029,-389.504014,0.370300,0.368710
7,0.171391,0.037561,100000.0,-3.954225,0.142924,-907.720645,-907.656488,0.166305,0.163684


In [84]:
pipe_lr_final = make_pipeline(column_transformer, Ridge(alpha=1000))
pipe_lr_final.fit(X_train, y_train)

eli5.explain_weights(pipe_lr_final.named_steps["ridge"], feature_names=new_columns)

Weight?,Feature
+2.206,number_of_reviews_ltm
+1.487,<BIAS>
+0.490,id
+0.381,last_review_month_10
+0.243,cozy
+0.231,last_review_month_11
+0.213,host_id
+0.138,number_of_reviews
+0.118,charming
+0.117,room_type_Private room


In [85]:
data={
    "Importance": pipe_lr_final.named_steps["ridge"].coef_,
}
pd.DataFrame(data=data, index=new_columns,).sort_values(
    by="Importance", ascending=False
)[:10]


,Importance
number_of_reviews_ltm,2.206021
id,0.489909
last_review_month_10,0.380700
cozy,0.243094
last_review_month_11,0.230811
host_id,0.213469
number_of_reviews,0.138209
charming,0.118373
room_type_Private room,0.116536
neighbourhood_group_Bronx,0.100016


#### Hyperparam Tune LGBM Regressor

In [128]:
param_lgbm = {
    'lgbmregressor__n_estimators': [10, 100, 1000],
    'lgbmregressor__max_depth': [5, 10, 15]
}


In [130]:
results_lgbm = {}

pipe_lgbm = make_pipeline(column_transformer, LGBMRegressor())
random_search_lgbm = RandomizedSearchCV(
    pipe_lgbm,
    param_lgbm,
    n_jobs=-1,
    return_train_score=True,
    scoring=scoring_metrics,
    refit='r2'
)

In [131]:
random_search_lgbm.fit(X_train_big, y_train_big)

/Users/valliakella/opt/miniconda3/envs/abnb/lib/python3.9/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('standardscaler',
                                                                               StandardScaler(),
                                                                               ['latitude',
                                                                                'longitude',
                                                                                'price',
                                                                                'minimum_nights',
                                                                                'calculated_host_listings_count',
                                                                                'availability_365',
                                                                                'last_review_year',
                                                     

In [133]:
pd.DataFrame(random_search_lgbm.cv_results_)[[
        "mean_fit_time", 
        "mean_score_time", 
        "param_lgbmregressor__n_estimators", 
        "param_lgbmregressor__max_depth",
        "mean_train_neg RMSE", 
        "std_train_neg RMSE",
        "mean_test_mape",
        "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by='mean_test_r2', ascending=False)

,mean_fit_time,mean_score_time,param_lgbmregressor__n_estimators,param_lgbmregressor__max_depth,mean_train_neg RMSE,std_train_neg RMSE,mean_test_mape,mean_train_mape,mean_test_r2,mean_train_r2
1,0.468376,0.068764,100,5,-2.496244,0.075635,-79.441512,-77.674748,0.461116,0.666349
4,0.465609,0.098433,100,10,-2.055569,0.060416,-69.720791,-65.301392,0.441955,0.773767
7,0.476878,0.113750,100,15,-1.954765,0.060678,-72.283469,-67.711531,0.431886,0.795426
0,0.370324,0.059154,10,5,-3.147709,0.087081,-586.921962,-586.435583,0.417316,0.469685
6,0.322518,0.057196,10,15,-3.032879,0.089289,-614.880257,-614.207838,0.417120,0.507745
3,0.307950,0.054652,10,10,-3.034432,0.088136,-613.182519,-612.257177,0.416696,0.507209
2,1.633706,0.295779,1000,5,-1.198019,0.073162,-105.511348,-87.052788,0.402606,0.922889
5,2.453600,0.729656,1000,10,-0.687727,0.061083,-97.150038,-70.598117,0.389419,0.974482
8,2.079207,0.719379,1000,15,-0.603113,0.043318,-88.721957,-61.366587,0.384043,0.980397


#### Hyperparam Tune CatBoostRegressor

## Stacking with Best Params from each model

In [140]:
regressors = {
    'ridge': pipe_poly,
    'catboost': make_pipeline(
        column_transformer, CatBoostRegressor(verbose=0)
    ),
    'randomforest': make_pipeline(
        column_transformer, RandomForestRegressor(random_state=123)
    ),
    'lgbm': make_pipeline(
        column_transformer, LGBMRegressor(verbose=0)
    )
}

stacking_reg = StackingRegressor(
    list(regressors.items()
        ), 
    final_estimator= Ridge()
)

results['stacking'] = cross_val_scores(
        stacking_reg,
        X_train,
        y_train,
        X_val,
        y_val,
        return_train_score=True
    )

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001813 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001783 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002736 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002770 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing

In [141]:
pd.DataFrame(results)

,Dummy,Ridge,Lasso,Random_Forest_reg,XGBoost_reg,lgbm_reg,catBoost_reg,poly_feat,stacking
r2_train,0.000000,0.389500,0.307888,0.926435,0.958081,0.795359,0.875625,0.550253,0.831034
r2_test,-0.000010,0.484823,0.407891,0.500858,0.433224,0.504185,0.553416,0.498116,0.563295
mse_train,18.968049,11.579991,13.128019,1.395379,0.795113,3.881636,2.359146,8.530831,3.204959
mse_test,17.962028,9.253537,10.635373,8.965511,10.180341,8.905761,8.021479,9.014759,7.844024
mape_train,14.434554,4.038569,9.018054,0.144263,0.603201,0.669306,0.841592,4.635072,1.470648
mape_test,14.117369,3.872633,8.823251,0.413916,0.915507,0.704105,0.916343,5.197266,1.700172
